In [ ]:
import requests
import requests
#Ne marche pas en notebook
#from top_20_artist_fr_list import artist_to_scrape  # Importer la liste d'artistes
artist_to_scrape = ['GIMS',
 'GIMS',
 'TIAKOLA',
 'GUY2BEZBAR',
 'LETO',
 'SDM',
 'CARBONNE',
 'BOOBA',
 'NINHO & NISKA',
 'SEVDALIZA, PABLLO VITTAR, YSEULT',
 'LACRIM',
 'LAYLOW',
 'AMK',
 'KEBLACK',
 'FLOYYMENOR, CRIS MJ',
 'KAROL G',
 'DISTURBED',
 'THEODORT',
 'BOUSS',
 'SABRINA CARPENTER']
# Votre clé d'API de Genius
GENIUS_API_TOKEN = 'vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG'

# Fonction pour obtenir l'ID d'un artiste
def search_artist(artist_name):
    base_url = "https://api.genius.com"
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + "/search"
    data = {'q': artist_name}
    response = requests.get(search_url, headers=headers, params=data)
    
    if response.status_code == 200:
        json_data = response.json()
        artist_hits = json_data['response']['hits']
        
        if artist_hits:
            for hit in artist_hits:
                if hit['result']['primary_artist']['name'].lower() == artist_name.lower():
                    return hit['result']['primary_artist']['id']
    else:
        print(f"Erreur lors de la recherche de l'artiste {artist_name}. Status code: {response.status_code}")
    return None

# Fonction pour récupérer toutes les chansons d'un artiste par ID
def get_artist_songs(artist_id, max_songs=100):
    base_url = f"https://api.genius.com/artists/{artist_id}/songs"
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    songs = []
    page = 1
    while True:
        params = {'page': page, 'per_page': 20}
        response = requests.get(base_url, headers=headers, params=params)
        
        if response.status_code == 200:
            json_data = response.json()
            song_list = json_data['response']['songs']
            
            if not song_list:
                break  # Si aucune chanson n'est retournée, arrêter la boucle
            
            for song in song_list:
                songs.append(song['title'])
                
            if len(songs) >= max_songs:
                break  # Limiter le nombre de chansons récupérées (par ex. 50)
            
            page += 1
        else:
            print(f"Erreur lors de la récupération des chansons. Status code: {response.status_code}")
            break

    return songs

# Fonction principale pour scraper les chansons de chaque artiste dans la liste
def scrape_artists_songs(artist_list, max_songs_per_artist=100):
    all_songs = {}
    
    for artist_name in artist_list:
        print(f"Recherche des chansons pour l'artiste : {artist_name}")
        artist_id = search_artist(artist_name)
        
        if artist_id:
            print(f"ID de l'artiste '{artist_name}': {artist_id}")
            songs = get_artist_songs(artist_id, max_songs=max_songs_per_artist)
            all_songs[artist_name] = songs
            print(f"Chansons de {artist_name} : {songs}")
        else:
            print(f"Artiste '{artist_name}' non trouvé.")
    
    return all_songs

# Exemple d'utilisation
all_artist_songs = scrape_artists_songs(artist_to_scrape, max_songs_per_artist=100)

# Vous pouvez maintenant utiliser all_artist_songs pour d'autres traitements ou l'enregistrer dans un fichier.


In [2]:
from lyricsgenius import Genius
import lyricsgenius as lg
import requests
token = 'vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG'


In [31]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os


GENIUS_API_TOKEN = 'vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG'
genius = lg.Genius(GENIUS_API_TOKEN)

def get_artist_id(artist_name):
    """Récupère l'ID d'un artiste en utilisant l'API Genius."""
    search_url = f"https://api.genius.com/search?q={artist_name}"
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    
    try:
        response = requests.get(search_url, headers=headers)
        response.raise_for_status()  # Vérifie les erreurs HTTP
        data = response.json()
        
        # Vérifie si la réponse contient la clé 'response'
        if 'response' in data and data['response']['hits']:
            return data['response']['hits'][0]['result']['primary_artist']['id']
        else:
            print(f"Aucune donnée trouvée pour l'artiste: {artist_name}")
            return None
    except Exception as e:
        print(f"Erreur lors de la récupération de l'ID pour {artist_name}: {e}")
        return None

def get_songs(artist_id, max_songs=140):
    """Récupère jusqu'à un nombre donné de chansons pour un artiste, triées par popularité."""
    songs = []
    page = 1
    
    while len(songs) < max_songs:
        url = f"https://api.genius.com/artists/{artist_id}/songs?page={page}"
        headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
        response = requests.get(url, headers=headers).json()
        songs_data = response['response']['songs']
        
        if not songs_data:
            break  # Sortir si aucune chanson n'est trouvée

        # Récupérer la popularité et ajouter les chansons à la liste
        for song in songs_data:
            if len(songs) < max_songs:  # Assurez-vous de ne pas dépasser max_songs
                songs.append({
                    'title': song['title'],
                    'popularity': song.get('stats', {}).get('pageviews', 0)  # Exemple de mesure de popularité
                })
            else:
                break  # Sortir de la boucle for si le maximum est atteint

        page += 1  # Passer à la page suivante

    # Trier manuellement par popularité
    sorted_songs = sorted(songs, key=lambda x: x['popularity'], reverse=True)
    return [song['title'] for song in sorted_songs[:max_songs]]  # Limiter le retour à max_songs

def download_lyrics(song_title, artist_name):
        """Télécharge les paroles d'une chanson donnée."""

        song = genius.search_song(song_title, artist_name)
        if song : 
            try :
                paroles = song.lyrics
            except Exception as e :
                print(f"Erreur lors de la récupération des paroles : {e}")
                return None
        
            return paroles, artist_name, song_title
        return None

import os

import os
import pandas as pd

def all_artist_songs(artist_to_scrape):
    """Télécharge les paroles de toutes les chansons pour chaque artiste et les stocke dans un fichier CSV par artiste."""
    for artist in artist_to_scrape:
        num = 1
        artist_id = get_artist_id(artist)
        if artist_id is not None:
            songs = get_songs(artist_id)
            if songs:
                # Créer un répertoire pour l'artiste si il n'existe pas
                os.makedirs(artist, exist_ok=True)
                artist_lyrics = []  # Liste pour stocker les données des chansons

                for song in songs:
                    lyrics_info = download_lyrics(song, artist)  # Modifiez cette ligne pour obtenir les infos des paroles
                    if lyrics_info:
                        lyrics, artist, song = lyrics_info
                        artist_lyrics.append({
                            'Artist': artist,
                            'Title': song,
                            'Lyrics': lyrics
                        })  # Ajouter les informations de la chanson
                        print(f"Téléchargé : {song} de {artist}")
                        print(f"Chanson numéro : {num}")
                        num +=1 
                    else:
                        print(f"Paroles non trouvées pour {song} de {artist}")

                # Créer un DataFrame à partir de la liste des paroles
                df = pd.DataFrame(artist_lyrics)

                # Enregistrer le DataFrame dans un fichier CSV
                df.to_csv(f"{artist}/{artist}_songs.csv", index=False, encoding='utf-8')
                print(f"Fichier créé : {artist}/{artist}_songs.csv")
            else:
                print(f"Aucune chanson trouvée pour {artist}")
        else:
            print(f"ID d'artiste non trouvé pour {artist}")

# Liste d'artistes à scraper
#   # Remplacez par votre liste d'artistes



In [ ]:
artist_to_scrape = ['GIMS',
 'TIAKOLA',
 'GUY2BEZBAR',
 'LETO',
 'SDM',
 'CARBONNE',
 'BOOBA',
 'NINHO & NISKA',
 'SEVDALIZA, PABLLO VITTAR, YSEULT',
 'LACRIM',
 'LAYLOW',
 'AMK',
 'KEBLACK',
 'FLOYYMENOR, CRIS MJ',
 'KAROL G',
 'DISTURBED',
 'THEODORT',
 'BOUSS',
 'SABRINA CARPENTER']


In [32]:
artist_to_scrape = ["ATEYABA"]

In [33]:
all_artist_songs(artist_to_scrape)

Searching for "Harajuku" by ATEYABA...


HTTPError: [Errno 429] 429 Client Error: Too Many Requests for url: https://genius.com/api/search/multi?q=Harajuku+ATEYABA

In [2]:
from lyricsgenius import Genius
import lyricsgenius as lg
import requests
import requests
import pandas as pd 
import numpy as np
import random 
from bs4 import BeautifulSoup

from tqdm import tqdm 
import os


token = 'vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG'
GENIUS_API_TOKEN = 'vEuhOt5S8b7JMcQuXZhlSNouoVIPQruY6a0bsie-lifgyPxsx_dPVzDSZools0pG'
genius = lg.Genius(GENIUS_API_TOKEN)
artist_to_scrape = ["BOOBA"]


class ArtistLyricsScraper:
    def __init__(self, GENIUS_API_TOKEN, artist_to_scrape) :
        self.GENIUS_API_TOKEN = GENIUS_API_TOKEN
        self.genius = lg.Genius(GENIUS_API_TOKEN)
        self.artist_to_scrape = artist_to_scrape
        
    def get_artist_id(self, artist_name):
        """Récupère l'ID d'un artiste en utilisant l'API Genius."""
        search_url = f"https://api.genius.com/search?q={artist_name}"
        headers = {'Authorization': 'Bearer ' + self.GENIUS_API_TOKEN}
        
        try:
            response = requests.get(search_url, headers=headers)
            response.raise_for_status()  # Vérifie les erreurs HTTP
            data = response.json()
            
            # Vérifie si la réponse contient la clé 'response'
            if 'response' in data and data['response']['hits']:
                return data['response']['hits'][0]['result']['primary_artist']['id']
            
            print(f"Aucune donnée trouvée pour l'artiste: {artist_name}")
            return None
        except Exception as e:
            print(f"Erreur lors de la récupération de l'ID pour {artist_name}: {e}")
            return None

    def get_songs(self, artist_id, max_songs=100):
        """Récupère jusqu'à un nombre donné de chansons pour un artiste, triées par popularité."""
        songs = []
        page = 1
        while len(songs) < max_songs:
            url = f"https://api.genius.com/artists/{artist_id}/songs?page={page}"
            headers = {'Authorization': 'Bearer ' + self.GENIUS_API_TOKEN}
            response = requests.get(url, headers=headers).json()
            songs_data = response['response']['songs']
            if not songs_data:
                break

            # Récupérer la popularité et trier manuellement (si disponible)
            for song in songs_data:
                if len(songs) >= max_songs:  # Stopper si on atteint la limite
                    break
                songs.append({
                    'title': song['title'],
                    'popularity': song.get('stats', {}).get('pageviews', 0)  # Exemple de mesure de popularité
                })

            page += 1

        # Trier manuellement par popularité
        sorted_songs = sorted(songs, key=lambda x: x['popularity'], reverse=True)
        return [song['title'] for song in sorted_songs]

    def download_lyrics(self, song_title, artist_name):
        """Télécharge les paroles d'une chanson donnée."""

        song = self.genius.search_song(song_title, artist_name)
        if song : 
            try :
                paroles = song.lyrics
            except Exception as e :
                return None
        
            return paroles
        return None

    def all_artist_songs(self, artist_to_scrape):
        """Télécharge les paroles de toutes les chansons pour 
        chaque artiste et les stocke dans un fichier par artiste."""
    for artist in artist_to_scrape:
        artist_id = self.get_artist_id(artist)
        if artist_id is not None:
            songs = get_songs(artist_id)
            if songs:
                # Créer un répertoire pour l'artiste si il n'existe pas
                os.makedirs(artist, exist_ok=True)
                artist_lyrics = []  # Liste pour stocker les paroles de toutes les chansons

                for song in songs:
                    lyrics = self.download_lyrics(song, artist)
                    if lyrics:
                        artist_lyrics.append(f"{song}\n{lyrics}\n")  # Ajouter les paroles au contenu de l'artiste
                        print(f"Téléchargé : {song} de {artist}")
                    else:
                        print(f"Paroles non trouvées pour {song} de {artist}")

                # Créer un fichier texte pour l'artiste contenant toutes les chansons
                with open(f"{artist}/{artist}_songs.txt", 'w', encoding='utf-8') as file:
                    file.write("\n".join(artist_lyrics))
                    print(f"Fichier créé : {artist}/{artist}_songs.txt")
            else:
                print(f"Aucune chanson trouvée pour {artist}")
        else:
            print(f"ID d'artiste non trouvé pour {artist}")

NameError: name 'self' is not defined

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Configuration de l'API Spotipy
client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'
credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=credentials)

# Fonction pour récupérer le genre d'une chanson
def get_song_genre(artist, title):
    # Recherche de l'artiste
    results = sp.search(q=f'artist:{artist} track:{title}', type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        # Récupérer l'ID de l'album pour obtenir le genre
        album_id = track['album']['id']
        album = sp.album(album_id)
        # Les genres sont dans la clé 'genres' de l'album
        return album['genres']  # Cela renvoie une liste de genres
    return None

# Itération dans le DataFrame et ajout des genres
df['Genre'] = df.apply(lambda row: get_song_genre(row['Artist'], row['Title']), axis=1)

# Affichage du DataFrame avec les genres
print(df)
